In [1]:
import numpy as np
import hFEMs as hs
import importlib
importlib.reload(hs);
# dir(hs)

In [ ]:
hs.print("0out","0err")
hs.print2("1out","1err")
hs.print("2out","2err")

In [ ]:
hs.print("0out","0err")
caps = hs.return_capsule_with_destructor()
caps2 = hs.return_capsule_with_destructor()
hs.print("1out","1err")
del caps
del caps2
hs.print("2out","2err")


In [ ]:
pairs = [(0,9),(1,8)]
CL = np.array([[1,0,0,0,0,0,0,0,0,0]],dtype=np.float)
dL = np.array([42],dtype=np.float)
ndof = 10
lengths = [1,2,3,4,5,6,7,8,9]
solvable = hs.TestSolvable(ndof, lengths, pairs, CL, dL)
print(solvable.grad(np.zeros(10,)))
ns = hs.NewtonSolver(solvable)



In [ ]:
ns

In [ ]:
# TO GRADIENT TESTING MODULE

%matplotlib inline
import matplotlib.pyplot as plt

def test_gradients(fun_val, fun_grad, q, dq, eps):
    fL = fun_val(q - eps * dq)
    fR = fun_val(q + eps * dq)
    dfdq = fun_grad(q)
    err = (fR - fL) - (2*eps * dfdq@dq)
    err = np.abs(err).max()
    return err

def plot_err(fun_val, fun_grad, ndims, epss, ndirs=1):
    q = np.random.uniform(-1,1,(ndims,))
    
    errs = np.empty((len(epss)),dtype=np.float)
    for k,eps in enumerate(epss):
        errmax = 0
        for i in range(ndirs):
            dq = np.random.uniform(-1,1,(ndims,))
            dq /= np.linalg.norm(dq)
            errmax = max(errmax, test_gradients(fun_val, fun_grad, q, dq, eps))
        errs[k] = errmax
    
    plt.plot(epss,errs,label="err")
    plt.plot(epss,epss,"--",label=r"$\varepsilon$")
    plt.plot(epss,epss**2,"--",label=r"$\varepsilon^2$")
    plt.plot(epss,epss**3,"--",label=r"$\varepsilon^3$")
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.show()

pairs = [(0,9),(1,8)]
CL = np.array([[1,0,0,0,0,0,0,0,0,0]],dtype=np.float)
dL = np.array([42],dtype=np.float)
tst = hs.TestSolvable(pairs, CL, dL)

plot_err(tst.val, tst.grad, ndims=10, epss=np.logspace(-10,0,50), ndirs=3)
plot_err(tst.grad, tst.hess, ndims=10, epss=np.logspace(-10,0,50), ndirs=3)